In [1]:
# import Glop package
!pip install ortools
from ortools.linear_solver import pywraplp as glp
from ortools.constraint_solver import pywrapcp as gcp
from ortools.constraint_solver import routing_enums_pb2

#from ortools.linear_solver import pywraplp as glp 

#Create IP model object
model = glp.Solver('Set Covering Problem', glp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

In [2]:
!pip install numpy
import numpy as np

In [3]:
np.__version__

'1.19.2'

The below are costs for traveling between locations using a bus and plane respectively. For simiplicity and a cleaner chunk of code, we calculated cost per route using distance and cost per mile in Excel

In [4]:
busdata=[[0, 493, 286, 289, 291, 413, 388, 465, 541, 538, 145],
        [499, 0, 417, 290, 278, 130, 169, 42, 48, 46, 619],
        [315, 454, 0, 141, 183, 314, 285, 411, 507, 504, 455],
        [310, 307, 138, 0, 44, 171, 143, 265, 358, 355, 463],
        [312, 294, 178, 44, 0, 158, 130, 252, 346, 343, 504],
        [411, 127, 284, 158, 147, 0, 40, 89, 176, 175, 542],
        [377, 162, 251, 130, 118, 39, 0, 124, 208, 206, 503],
        [396, 35, 317, 210, 200, 77, 108, 0, 76, 74, 504],
        [542, 48, 461, 335, 324, 178, 216, 89, 0, 5, 671],
        [538, 45, 458, 332, 320, 176, 212, 87, 5, 0, 666],
        [163, 685, 462, 484, 527, 611, 581, 664, 749, 746, 0]]


planedata=[[0, 459, 329, 331, 333, 409, 393, 442, 489, 488, 241],
          [459, 0, 409, 329, 322, 230, 254, 176, 180, 179, 534], 
          [329, 409, 0, 230, 254, 329, 312, 384, 438, 437, 409], 
          [331, 329, 230, 0, 176, 250, 234, 305, 359, 358, 421], 
          [333, 322, 254, 176, 0, 243, 226, 298, 352, 351, 445], 
          [409, 230, 329, 250, 243, 0, 176, 206, 261, 260, 492], 
          [393, 254, 312, 234, 226, 176, 0, 230, 285, 283, 475], 
          [442, 176, 384, 305, 298, 206, 230, 0, 206, 204, 522], 
          [489, 180, 438, 359, 352, 261, 285, 206, 0, 153, 569],
          [488, 179, 437, 358, 351, 260, 283, 204, 153, 0, 567],
          [241, 534, 409, 421, 445, 492, 475, 522, 569, 568, 0]]


busdata
planedata


[[0, 459, 329, 331, 333, 409, 393, 442, 489, 488, 241],
 [459, 0, 409, 329, 322, 230, 254, 176, 180, 179, 534],
 [329, 409, 0, 230, 254, 329, 312, 384, 438, 437, 409],
 [331, 329, 230, 0, 176, 250, 234, 305, 359, 358, 421],
 [333, 322, 254, 176, 0, 243, 226, 298, 352, 351, 445],
 [409, 230, 329, 250, 243, 0, 176, 206, 261, 260, 492],
 [393, 254, 312, 234, 226, 176, 0, 230, 285, 283, 475],
 [442, 176, 384, 305, 298, 206, 230, 0, 206, 204, 522],
 [489, 180, 438, 359, 352, 261, 285, 206, 0, 153, 569],
 [488, 179, 437, 358, 351, 260, 283, 204, 153, 0, 567],
 [241, 534, 409, 421, 445, 492, 475, 522, 569, 568, 0]]

The optimal route is going to be one with the lowest cost. In order to find the optimal option for travel between each university, we need to find the lowest cost for each leg of the journey between both methods. We created a new minimum cost matrix below to achieve that

In [5]:
temp1=np.less_equal(busdata,planedata)
temp2=np.less(planedata,busdata)
C1=np.multiply(busdata,temp1)
C2=np.multiply(planedata,temp2)
C=C1+C2
print(C)

[[  0 459 286 289 291 409 388 442 489 488 145]
 [459   0 409 290 278 130 169  42  48  46 534]
 [315 409   0 141 183 314 285 384 438 437 409]
 [310 307 138   0  44 171 143 265 358 355 421]
 [312 294 178  44   0 158 130 252 346 343 445]
 [409 127 284 158 147   0  40  89 176 175 492]
 [377 162 251 130 118  39   0 124 208 206 475]
 [396  35 317 210 200  77 108   0  76  74 504]
 [489  48 438 335 324 178 216  89   0   5 569]
 [488  45 437 332 320 176 212  87   5   0 567]
 [163 534 409 421 445 492 475 522 569 568   0]]


In [6]:
type(C)

numpy.ndarray

In [7]:
AC= C.tolist()

In [8]:
AC

[[0, 459, 286, 289, 291, 409, 388, 442, 489, 488, 145],
 [459, 0, 409, 290, 278, 130, 169, 42, 48, 46, 534],
 [315, 409, 0, 141, 183, 314, 285, 384, 438, 437, 409],
 [310, 307, 138, 0, 44, 171, 143, 265, 358, 355, 421],
 [312, 294, 178, 44, 0, 158, 130, 252, 346, 343, 445],
 [409, 127, 284, 158, 147, 0, 40, 89, 176, 175, 492],
 [377, 162, 251, 130, 118, 39, 0, 124, 208, 206, 475],
 [396, 35, 317, 210, 200, 77, 108, 0, 76, 74, 504],
 [489, 48, 438, 335, 324, 178, 216, 89, 0, 5, 569],
 [488, 45, 437, 332, 320, 176, 212, 87, 5, 0, 567],
 [163, 534, 409, 421, 445, 492, 475, 522, 569, 568, 0]]

Due to compatibility issues between the Numpy data array and Google's OR Tools package, we converted the numpy array into a list to allow it to be called into our data model and create an automated model.

In [9]:
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['cost_matrix'] = AC   
    data['num_vehicles'] = 1
    data['depot'] = 0
    return data

data = create_data_model()
data

{'cost_matrix': [[0, 459, 286, 289, 291, 409, 388, 442, 489, 488, 145],
  [459, 0, 409, 290, 278, 130, 169, 42, 48, 46, 534],
  [315, 409, 0, 141, 183, 314, 285, 384, 438, 437, 409],
  [310, 307, 138, 0, 44, 171, 143, 265, 358, 355, 421],
  [312, 294, 178, 44, 0, 158, 130, 252, 346, 343, 445],
  [409, 127, 284, 158, 147, 0, 40, 89, 176, 175, 492],
  [377, 162, 251, 130, 118, 39, 0, 124, 208, 206, 475],
  [396, 35, 317, 210, 200, 77, 108, 0, 76, 74, 504],
  [489, 48, 438, 335, 324, 178, 216, 89, 0, 5, 569],
  [488, 45, 437, 332, 320, 176, 212, 87, 5, 0, 567],
  [163, 534, 409, 421, 445, 492, 475, 522, 569, 568, 0]],
 'num_vehicles': 1,
 'depot': 0}

In [10]:
data = create_data_model()
manager = gcp.RoutingIndexManager(len(data['cost_matrix']),data['num_vehicles'], data['depot'])
routing = gcp.RoutingModel(manager)

In [11]:
def cost_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['cost_matrix'][from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(cost_callback)

In [12]:
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

In [13]:
search_parameters = gcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

In [14]:
def print_solution(manager, routing, solution):
    """Prints solution on console."""
    print('Lowest Cost: ${}'.format(solution.ObjectiveValue()))
    index = routing.Start(0)
    plan_output = 'The best route for the University of Cincinnati to travel in the Big 12 is:\n'
    route_cost = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_cost += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    print(plan_output)
    plan_output += 'Route cost: $ {}\n'.format(route_cost)

In [17]:
solution = routing.SolveWithParameters(search_parameters)
if solution:
    print_solution(manager, routing, solution)

Lowest Cost: $1538
The best route for the University of Cincinnati to travel in the Big 12 is:
 0 -> 10 -> 8 -> 9 -> 1 -> 7 -> 5 -> 6 -> 4 -> 3 -> 2 -> 0



In [16]:
#0 -bus> 10 -plane> 8 -bus> 9 -bus> 1 -bus> 7 -bus> 5 -bus> 6 -bus> 4 -bus> 3 -bus> 2 -bus> 0
#0 -145> 10 -569> 8 -5> 9 -45> 1 -42> 7 -77> 5 -40> 6 -118> 4 -44> 3 -137> 2 -315> 0